In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import functools
from traitlets import traitlets

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn import tree
import plotly.express as px

# Declaring Functions

## Model Inits/Plot Info

In [ ]:
def init_nn(activation='tanh', alpha=0.001, l1_size=8, l2_size=4, l3_size=0, l4_size=0, lr=0.001):
    h_layer_sizes = (l1_size,)
    
    if l2_size != 0 and l3_size == 0 and l4_size == 0:
        h_layer_sizes = (l1_size, l2_size)
    elif l2_size != 0 and l3_size != 0 and l4_size == 0:
        h_layer_sizes = (l1_size, l2_size, l3_size)
    elif l2_size != 0 and l3_size != 0 and l4_size != 0:
        h_layer_sizes = (l1_size, l2_size, l3_size, l4_size)
            
    clf = MLPClassifier(activation=activation, solver='adam', alpha=alpha, hidden_layer_sizes=h_layer_sizes,
                    learning_rate="constant", learning_rate_init=lr,
                    random_state=42, max_iter=1, warm_start=True)
    return clf

def gradient_descent(X, y, lr=0.001, epoch=300, lmd = 0.01, reg='L2'):
    b1, b0 = 0.0, 0.0 # parameters
    b1_list, b0_list = [], []
    log, mse = [], [] # lists to store learning process
    
    for i in range(epoch):
        sumyhat = 0
        sumxyhat = 0

        reg_grad = 0

        if reg == 'L1':
            reg_grad = lmd
        elif reg == 'L2':
            reg_grad = 2*b1*lmd

        for j in range(len(X)):
            sumyhat += b0 + b1*X[j] + reg_grad - y[j]
            sumxyhat += (b0 + b1*X[j] + reg_grad - y[j])*(X[j])
            
        # CALCULATE AND UPDATE b1 AND b0
        b1 -= lr*(1/len(X))*sumxyhat
        b0 -= lr*(1/len(X))*sumyhat
        b1_list.append(b1)
        b0_list.append(b0)

        log.append((b1, b0))
        reg_term = 0

        if reg == 'L1':
            reg_term = lmd*np.sum(b1)
        elif reg == 'L2':
            reg_term = lmd*np.sum(b1*b1)

        mse.append(mean_squared_error(y, (b1*X + b0)) + reg_term)        

    return b1, b0, log, mse, b1_list, b0_list


def init_lr(alpha=0.001, lr=0.001, reg_type="L2"):
            
    clf = SGDClassifier(loss="log", penalty=reg_type.lower(), alpha=alpha, learning_rate="constant",
                    eta0=lr, max_iter=1, warm_start=True)
    return clf


def init_lsvm(alpha=0.001, lr=0.001, reg_type="L2"):
          
    clf = SGDClassifier(loss="hinge", penalty=reg_type.lower(), alpha=alpha, learning_rate="constant",
                    eta0=lr, max_iter=1, warm_start=True)
    return clf


def kmeans_cluster_centers(X, xx, yy, n_clusters=2, iterations=100):
    n_centroids = []
    labels = []
    Z_l = []
    centroids = None
    for i in range(iterations):
        clus = KMeans(
            max_iter=1,
            n_init=1,
            init=(centroids if centroids is not None else 'k-means++'),
            n_clusters=n_clusters,
            random_state=1)
        clus.fit(X)
        centroids = clus.cluster_centers_
        n_centroids.append(centroids)
        labels.append(clus.fit_predict(X))
        Z_l.append(clus.predict(np.c_[xx.ravel(), yy.ravel()]))
    return n_centroids, labels, Z_l


def init_gnb():
    
    clf = GaussianNB()
    return clf


def init_dtree(criterion='gini'):
    
    clf = DecisionTreeClassifier(criterion=criterion)
    return clf
    

## Dataset Inits

In [ ]:
def init_nn_dataset(dataset="Circular"):
    if dataset == "Circular":
        df = pd.read_csv("classification_circle.csv")
        df.loc[ df["y"] == -1, "y"] = 0 
    elif dataset == "XOR":
        df = pd.read_csv("classification_nonlinear_xor.csv")
        df.loc[ df["y"] == -1, "y"] = 0
    
    feature_cols = ['x1', 'x2']
    X = df[feature_cols].to_numpy()
    y = df['y'].to_numpy()
    
    return X, y, df


def init_linreg_dataset(dataset="Linear"):
    if dataset == "Linear":
        df = pd.read_csv("regression_linear_line.csv")
    elif dataset == "Square Root":
        df = pd.read_csv("regression_linear_square_root.csv")    
    
    X = np.array(df['x'])
    y = np.array(df['y'])
    
    return X, y, df


def init_lr_dataset(dataset="Uniform"):
    if dataset == "Uniform":
        df = pd.read_csv("classification_linear_uniform.csv")
        df.loc[ df["y"] == -1, "y"] = 0  
    elif dataset == "XOR":
        df = pd.read_csv("classification_nonlinear_xor.csv")
        df.loc[ df["y"] == -1, "y"] = 0
    
    feature_cols = ['x1', 'x2']
    X = df[feature_cols].to_numpy()
    y = df['y'].to_numpy()
    
    return X, y, df


def init_lsvm_dataset(dataset="Uniform"):
    if dataset == "Uniform":
        df = pd.read_csv("classification_linear_uniform.csv")
        df.loc[ df["y"] == -1, "y"] = 0  
    elif dataset == "XOR":
        df = pd.read_csv("classification_nonlinear_xor.csv")
        df.loc[ df["y"] == -1, "y"] = 0
    
    feature_cols = ['x1', 'x2']
    X = df[feature_cols].to_numpy()
    y = df['y'].to_numpy()
    
    return X, y, df


def init_kmeans_dataset(dataset='4 Cluster'):
    if dataset == '4 Cluster':
        df = pd.read_csv("clustering_4clusters.csv")
    elif dataset == 'Uniform':
        df = pd.read_csv("classification_linear_uniform.csv")
    elif dataset == 'XOR':
        df = pd.read_csv("classification_nonlinear_xor.csv")
    elif dataset == 'With Outliers':
        df = pd.read_csv("classification_clustering_outliers.csv")
    
    feature_cols = ['x1', 'x2']
    X = df[feature_cols].to_numpy()
    
    return X, df


def init_gnb_dataset(dataset="Independent"):
    if dataset == "Independent":
        df = pd.read_csv("classification_naive_bayes_bernoulli_independent.csv")
        df.loc[ df["y"] == -1, "y"] = 0 
    elif dataset == "Dependent":
        df = pd.read_csv("classification_dependent_feature.csv")
        df.loc[ df["y"] == -1, "y"] = 0 
    
    feature_cols = ['x1', 'x2']
    X = df[feature_cols].to_numpy()
    y = df['y'].to_numpy()
    
    return X, y, df


def init_dtree_dataset(dataset="XOR"):
    if dataset == "Uniform":
        df = pd.read_csv("classification_linear_uniform.csv")
        df.loc[ df["y"] == -1, "y"] = 0  
    elif dataset == "XOR":
        df = pd.read_csv("classification_nonlinear_xor.csv")
        df.loc[ df["y"] == -1, "y"] = 0
    
    feature_cols = ['x1', 'x2']
    X = df[feature_cols].to_numpy()
    y = df['y'].to_numpy()
    
    return X, y, df

## Animations

In [ ]:
def animate_nn(i, *fargs):
    clf, X, y, df, xx, yy = fargs
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    clf.fit(X, y)
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.cla()
    plt.imshow(
    Z,
    interpolation="nearest",
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    aspect="auto",
    origin="lower",
    alpha = 0.7,
    cmap=plt.cm.Paired
    )
    contours = plt.contour(xx, yy, Z, linewidths=2, colors="white")
    sns.scatterplot(x="x1", y="x2", hue="y", data=df)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.title("Iteration: {}".format(i))
    plt.legend(loc='lower right')
    
    
def animate_linreg(i, *fargs):
    X, y, w_list, b_list = fargs 
    
    plt.cla()
    sns.scatterplot(x=X, y=y)
    y_graph = w_list[i-1]*X + b_list[i-1]
    plt.title("Iteration: {}".format(i))
    plt.plot(X, y_graph, 'r', linewidth=2.5)

    
def animate_lr(i, *fargs):
    clf, X, y, df = fargs
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    clf.fit(X, y)
    
    b = clf.intercept_[0]
    w1, w2 = clf.coef_.T
    b = -b/w2
    theta = -w1/w2

    xd = np.array([xmin, xmax])
    yd = theta*xd + b
    
    plt.cla()
    plt.plot(xd, yd, 'w', lw=3)
    plt.fill_between(xd, yd, ymin, color='tab:blue', alpha=0.4)
    plt.fill_between(xd, yd, ymax, color='tab:orange', alpha=0.4)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    sns.scatterplot(x="x1", y="x2", hue="y", data=df)
    plt.title("Iteration: {}".format(i))
    plt.legend(loc='lower right')

    
def animate_lsvm(i, *fargs):
    clf, X, y, df = fargs
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    clf.fit(X, y)
    
    b = clf.intercept_[0]
    w1, w2 = clf.coef_.T
    b = -b/w2
    theta = -w1/w2

    xd = np.array([xmin, xmax])
    yd = theta*xd + b
    
    plt.cla()
    plt.plot(xd, yd, 'w', lw=3)
    plt.fill_between(xd, yd, ymin, color='tab:blue', alpha=0.4)
    plt.fill_between(xd, yd, ymax, color='tab:orange', alpha=0.4)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    sns.scatterplot(x="x1", y="x2", hue="y", data=df)
    plt.title("Iteration: {}".format(i))
    plt.legend(loc='lower right')


def animate_kmeans(i, *fargs):
    n_centroids, labels, Z_l, df, xx, yy, inter = fargs
    
    plt.cla()
    Z = Z_l[i].reshape(xx.shape)
    plt.imshow(
    Z,
    interpolation=inter,
    extent=(xx.min(), xx.max(), yy.min(), yy.max()),
    cmap=plt.cm.Paired,
    alpha=0.5,
    aspect="auto",
    origin="lower",
    )
    sns.scatterplot(x="x1", y="x2", hue=labels[i], palette="deep", data=df)
    centroids = n_centroids[i]
    plt.scatter(
        centroids[:, 0],
        centroids[:, 1],
        marker="x",
        s=80,
        linewidths=3,
        color="black",
        zorder=10,
    )
    plt.title("Iteration: {}".format(i))
    plt.legend(loc='lower right')
        

## Plays/Plots

In [ ]:
def play_nn(btn_state, activation='tanh', alpha=0.001, l1_size=8, l2_size=4, l3_size=0,
            l4_size=0, lr=0.001, dataset='Circular'):
    
    btn_state.disabled = True
    anim_running = True
    
    def onClick(event): # start/stop animation when clicked on figure 
        nonlocal anim_running
        if anim_running:
            anim.event_source.stop()
            anim_running = False
        else:
            anim.event_source.start()
            anim_running = True
    
    plt.clf() # clear figure  
    clf = init_nn(activation=activation, alpha=alpha, l1_size=l1_size, l2_size=l2_size, l3_size=l3_size,
                  l4_size=l4_size, lr=lr)    
    X, y, df = init_nn_dataset(dataset=dataset)
    
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(xmin, xmax, 500))
    plt.gcf().canvas.mpl_connect('button_press_event', onClick) # connects the figure with the onClick func
    anim = FuncAnimation(plt.gcf(), func=animate_nn, fargs=(clf, X, y, df, xx, yy), interval=100,
                         frames=200, blit=True)
    btn_state.value = anim
    return anim


def play_linreg(btn_state, alpha=0.001, reg_type="L2", lr=0.001, dataset='Linear', epoch=300):
    
    btn_state.disabled = True
    anim_running = True
    
    def onClick(event): # start/stop animation when clicked on figure 
        nonlocal anim_running
        if anim_running:
            anim.event_source.stop()
            anim_running = False
        else:
            anim.event_source.start()
            anim_running = True
    
    plt.clf()
    X, y, df = init_linreg_dataset(dataset=dataset)
    w, b, _, _, w_list, b_list = gradient_descent(X=X, y=y, lr=lr, epoch=epoch, lmd=alpha,
                                                  reg=reg_type)
    
    plt.gcf().canvas.mpl_connect('button_press_event', onClick) # connects the figure with the onClick func
    anim = FuncAnimation(plt.gcf(), func=animate_linreg, fargs=(X, y, w_list, b_list),
                        repeat=False, frames=epoch, interval=20)
    btn_state.value = anim
    return anim


def play_lr(btn_state, alpha=0.001, reg_type="L2", lr=0.001, dataset='Uniform'):
    
    btn_state.disabled = True
    anim_running = True

    def onClick(event): # start/stop animation when clicked on figure 
        nonlocal anim_running
        if anim_running:
            anim.event_source.stop()
            anim_running = False
        else:
            anim.event_source.start()
            anim_running = True
        
    plt.clf() # clear figure 
    clf = init_lr(alpha=alpha, lr=lr, reg_type=reg_type)
    X, y, df = init_lr_dataset(dataset=dataset)
    plt.gcf().canvas.mpl_connect('button_press_event', onClick) # connects the figure with the onClick func
    anim = FuncAnimation(plt.gcf(), func=animate_lr, fargs=(clf, X, y, df), interval=100,
                         frames=200, blit=True)
    btn_state.value = anim
    return anim


def play_lsvm(btn_state, alpha=0.001, reg_type="L2", lr=0.001, dataset='Uniform'):
    
    btn_state.disabled = True
    anim_running = True

    def onClick(event): # start/stop animation when clicked on figure 
        nonlocal anim_running
        if anim_running:
            anim.event_source.stop()
            anim_running = False
        else:
            anim.event_source.start()
            anim_running = True
        
    plt.clf() # clear figure 
    clf = init_lsvm(alpha=alpha, lr=lr, reg_type=reg_type)
    X, y, df = init_lsvm_dataset(dataset=dataset)
    plt.gcf().canvas.mpl_connect('button_press_event', onClick) # connects the figure with the onClick func
    anim = FuncAnimation(plt.gcf(), func=animate_lsvm, fargs=(clf, X, y, df), interval=100,
                         frames=200, blit=True)
    btn_state.value = anim
    return anim


def play_kmeans(btn_state, n_clusters=3, dataset='4 Cluster'):
    
    btn_state.disabled = True
    anim_running = True
    
    def onClick(event): # start/stop animation when clicked on figure 
        nonlocal anim_running
        if anim_running:
            anim.event_source.stop()
            anim_running = False
        else:
            anim.event_source.start()
            anim_running = True
    
    plt.clf() # clear figure     
    X, df = init_kmeans_dataset(dataset=dataset)
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(ymin, ymax, 500))
    
    plt.gcf().canvas.mpl_connect('button_press_event', onClick) # connects the figure with the onClick func
    n_centroids, labels, Z_l = kmeans_cluster_centers(X, xx, yy, n_clusters=n_clusters, iterations=50)
    anim = FuncAnimation(plt.gcf(),
                            repeat=False,
                            func = animate_kmeans,
                            fargs=(n_centroids, labels, Z_l, df, xx, yy, "none"),
                            frames = 100,
                            interval = 800)
    btn_state.value = anim
    return anim


def plot_gnb(btn_state, dataset='Independent'):
    plt.clf()
    X, y, df = init_gnb_dataset(dataset=dataset)
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    
    clf = init_gnb()
    clf.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(ymin, ymax, 500))
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])
    Z = Z[:, 1].reshape(xx.shape)

    plt.imshow(
        Z,
        interpolation="nearest",
        extent=(xx.min(), xx.max(), yy.min(), yy.max()),
        aspect="auto",
        origin="lower",
        cmap=plt.cm.PuOr_r,
        alpha=0.7
    )

    contours = plt.contour(xx, yy, Z, linewidths=1, colors="white", alpha=0.1)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    sns.scatterplot(x="x1", y="x2", hue="y", data=df)
    plt.legend(loc='lower right')
    plt.show()
    

def plot_dtree(btn_state, criterion='entropy', dataset='XOR'):
    plt.clf()
    X, y, df = init_dtree_dataset(dataset=dataset)
    xmin, xmax = df["x1"].min(), df["x1"].max() 
    ymin, ymax = df["x2"].min(), df["x2"].max()
    
    clf = init_dtree(criterion=criterion)
    clf.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, 500), np.linspace(ymin, ymax, 500))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.imshow(
            Z,
            interpolation="nearest",
            extent=(xx.min(), xx.max(), yy.min(), yy.max()),
            aspect="auto",
            origin="lower",
            cmap=plt.cm.Paired,
            alpha=0.7
        )

    contours = plt.contour(xx, yy, Z, linewidths=2, colors="white", alpha=1)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    sns.scatterplot(x="x1", y="x2", hue="y", data=df)
    plt.legend(loc='lower right')
    plt.show()
    fig = plt.figure(figsize=(8,4))
    _ = tree.plot_tree(clf, 
                       filled=True)

## Tests

In [ ]:
def test(btn_state, activation='tanh', alpha=0.001, l1_size=8, l2_size=4, l3_size=0,
            l4_size=0, lr=0.001, dataset='Circular'):
    print(activation, alpha, l1_size, l2_size, l3_size, l4_size, lr, dataset)

In [ ]:
def init_logreg(a, c, d):
    return a, c, d

def animate_logreg(a, c, d):
    print(a, c, d)

def on_button_clicked_2(b, a, c, d):
    with output:
        e, f, g = init_logreg(a, c, d)
        animate_logreg(e, f, g)

# Declaring Widgets

In [ ]:
class LoadedButton(widgets.Button):
    """A custom button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

In [ ]:
algorithms = ['Linear Regression', 'Logistic Regression', 'Neural Network', 'Linear SVM', 'Non-linear SVM', 'KMeans',
             'Naive Bayes', 'Decision Tree', 'PCA', 'None']

output = widgets.Output()

algo_dropdown = widgets.Dropdown(
            options=algorithms,
            value='None',
            description='Algorithm:',
            disabled=False,
        )

activation_dropdown = widgets.Dropdown(
            options=['logistic', 'tanh', 'relu'],
            value='tanh',
            description='Activation:',
            disabled=False,
        )

lr_slider = widgets.FloatSlider(
            value=0.001,
            min=0.0001,
            max=0.1,
            step=0.0001,
            description='alpha:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.4f',
        )

reg_strength_slider = widgets.FloatSlider(
            value=0.001,
            min=0.0001,
            max=0.1,
            step=0.0001,
            description='lambda:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.4f',
        )

regtype_dropdown = widgets.Dropdown(
            options=['L1', 'L2', 'None'],
            value='L2',
            description='Reg:',
            disabled=False,
        )

n_clusters_slider = widgets.IntSlider(
            value=3,
            min=1,
            max=7,
            step=1,
            description='No. of Clusters:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )

dtree_criterion_dropdown = widgets.Dropdown(
            options=['gini', 'entropy'],
            value='entropy',
            description='Dataset',
            disabled=False,
        )

l1_dropdown = widgets.Dropdown(
            options=[1, 2, 4, 8],
            value=8,
            description='L1 Size:',
            disabled=False,
        )

l2_dropdown = widgets.Dropdown(
            options=[0, 1, 2, 4, 8],
            value=4,
            description='L2 Size:',
            disabled=False,
        )

l3_dropdown = widgets.Dropdown(
            options=[0, 1, 2, 4, 8],
            value=0,
            description='L3 Size:',
            disabled=False,
        )

l4_dropdown = widgets.Dropdown(
            options=[0, 1, 2, 4, 8],
            value=0,
            description='L4 Size:',
            disabled=False,
        )

dataset_nn_dropdown = widgets.Dropdown(
            options=['Circular', 'XOR'],
            value='Circular',
            description='Dataset',
            disabled=False,
        )

dataset_linreg_dropdown = widgets.Dropdown(
            options=['Linear', 'Square Root'],
            value='Square Root',
            description='Dataset',
            disabled=False,
        )

dataset_lr_dropdown = widgets.Dropdown(
            options=['Uniform', 'XOR'],
            value='XOR',
            description='Dataset',
            disabled=False,
        )

dataset_lsvm_dropdown = widgets.Dropdown(
            options=['Uniform', 'XOR'],
            value='XOR',
            description='Dataset',
            disabled=False,
        )

dataset_kmeans_dropdown = widgets.Dropdown(
            options=['Uniform', 'XOR', 'With Outliers', '4 Cluster'],
            value='4 Cluster',
            description='Dataset',
            disabled=False,
        )

dataset_gnb_dropdown = widgets.Dropdown(
            options=['Independent', 'Dependent'],
            value='Independent',
            description='Dataset',
            disabled=False,
        )

dataset_dtree_dropdown = widgets.Dropdown(
            options=['Uniform', 'XOR'],
            value='XOR',
            description='Dataset',
            disabled=False,
        )

play_nn_btn = LoadedButton(icon="play")
play_lr_btn = LoadedButton(icon="play")
play_linreg_btn = LoadedButton(icon="play")
play_lsvm_btn = LoadedButton(icon="play")
play_kmeans_btn = LoadedButton(icon="play")
plot_gnb_btn = LoadedButton(description="Plot")
plot_dtree_btn = LoadedButton(description="Plot")

# Main 

In [ ]:
# %matplotlib notebook
# %matplotlib notebook
%matplotlib widget
%matplotlib widget
def run(algo='None'):
    if algo == "Linear Regression":
        play_linreg_btn.disabled = False
        hb = widgets.HBox([lr_slider, reg_strength_slider])
        vb = widgets.VBox([regtype_dropdown, hb, dataset_linreg_dropdown, play_linreg_btn, output])
        display(vb)
        play_linreg_btn.on_click(functools.partial(play_linreg,
                                            alpha=reg_strength_slider.value,
                                            reg_type=regtype_dropdown.value,
                                            lr=lr_slider.value,
                                            dataset=dataset_linreg_dropdown.value))
        return play_linreg_btn.value
    
    elif algo == "Logistic Regression":
        play_lr_btn.disabled = False
        hb = widgets.HBox([lr_slider, reg_strength_slider])
        vb = widgets.VBox([regtype_dropdown, hb, dataset_lr_dropdown, play_lr_btn, output])
        display(vb)
        play_lr_btn.on_click(functools.partial(play_lr,
                                            alpha=reg_strength_slider.value,
                                            reg_type=regtype_dropdown.value,
                                            lr=lr_slider.value,
                                            dataset=dataset_lr_dropdown.value))
        
        return play_lr_btn.value
        
    elif algo == "Neural Network":
        play_nn_btn.disabled = False
        hb1 = widgets.HBox([lr_slider, reg_strength_slider])
        hb2 = widgets.HBox([l1_dropdown, l2_dropdown, l3_dropdown, l4_dropdown])
        vb = widgets.VBox([activation_dropdown, hb1, hb2, dataset_nn_dropdown, play_nn_btn, output])
        display(vb)
        play_nn_btn.on_click(functools.partial(play_nn, activation=activation_dropdown.value,
                                            alpha=reg_strength_slider.value,
                                            l1_size=l1_dropdown.value,
                                            l2_size=l2_dropdown.value, l3_size=l3_dropdown.value,
                                            l4_size=l4_dropdown.value, lr=lr_slider.value,
                                            dataset=dataset_nn_dropdown.value))
        return play_nn_btn.value
    
    elif algo == "Linear SVM":
        play_lsvm_btn.disabled = False
        hb = widgets.HBox([lr_slider, reg_strength_slider])
        vb = widgets.VBox([regtype_dropdown, hb, dataset_lsvm_dropdown, play_lsvm_btn, output])
        display(vb)
        play_lsvm_btn.on_click(functools.partial(play_lsvm,
                                            alpha=reg_strength_slider.value,
                                            reg_type=regtype_dropdown.value,
                                            lr=lr_slider.value,
                                            dataset=dataset_lsvm_dropdown.value))
        return play_lsvm_btn.value
    
    elif algo == "KMeans":
        play_kmeans_btn.disabled = False
        vb = widgets.VBox([dataset_kmeans_dropdown, n_clusters_slider, play_kmeans_btn, output])
        display(vb)
        play_kmeans_btn.on_click(functools.partial(play_kmeans,
                                                   n_clusters=n_clusters_slider.value,
                                                   dataset=dataset_kmeans_dropdown.value))
        return play_kmeans_btn.value
    
    elif algo == "Naive Bayes":
        vb = widgets.VBox([dataset_gnb_dropdown, plot_gnb_btn, output])
        display(vb)
        plot_gnb_btn.on_click(functools.partial(plot_gnb,
                                                   dataset=dataset_gnb_dropdown.value))
    
    elif algo == "Decision Tree":
        vb = widgets.VBox([dataset_dtree_dropdown, dtree_criterion_dropdown, plot_dtree_btn, output])
        display(vb)
        plot_dtree_btn.on_click(functools.partial(plot_dtree,
                                                  criterion=dtree_criterion_dropdown.value,
                                                  dataset=dataset_dtree_dropdown.value))

In [ ]:
widgets.interact(run, algo=algo_dropdown);